## assignment 1

In [1]:
import xml.etree.ElementTree as ET
from stemming.porter2 import stem
import re

### load stop word

In [2]:
stopword_path = 'englishST.txt'
def load_stopword(stopword_path):
    stopword_list = []
    with open(stopword_path, 'r',encoding='utf-8') as f1:
        stopword_list = [str(current_word).strip() for current_word in f1]
    return stopword_list

stopword_list = load_stopword(stopword_path)

In [3]:
def tokenisation_text(text):
    
    # simply remove every non-letter character
    #     del_punc = r'[^A-Za-z0-9_-]' # keep _ -
    #     del_punc = r'[^A-Za-z0-9_-\']' #keep _ - '
    del_punc = r'[\W]' # keep _
    
    text_nopunc = re.sub('&amp',' ',text)
    text_nopunc = re.sub(del_punc,' ',text)
    
    tokenisation_list = text_nopunc.split()
    return tokenisation_list

def lower_word(word_list):
    return [current_word.strip().lower() for current_word in word_list]

def token_lower_nostop_stem_list(all_text, stopword_list):
    token_list = tokenisation_text(all_text)
    token_lowerlist = lower_word(token_list)
    token_lowerlist_nostop = [str(current_word) for current_word in token_lowerlist if str(current_word) not in stopword_list]
    stem_list = [stem(current_word) for current_word in token_lowerlist_nostop]

    return stem_list

In [4]:
def to_standard_xml(xml_file_path):
    new_file_name = 'standard_' + xml_file_path.split('/')[-1]
    new_file_path = data_folder + new_file_name

    with open(new_file_path,'w',encoding='utf-8') as new_file:
        with open(xml_file_path,'r') as original_file:
            new_file.write('<?xml version="1.0"?>\n')
            new_file.write('<sample>\n')
            for line in original_file:
                if(len(line.strip()) == 0):
                    continue
                new_file.write(line)
            new_file.write('</sample>')
    return new_file_path

def merge_text(text1,text2):
    merged_text = text1 + text2
    return merged_text

def xml_all_text(xml_file_path,stopword_list,is_sample):
    if (is_sample):
        # sample
        standard_xml_path = to_standard_xml(sample_xml_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text,token_lower_nostop_stem_list(child.find('Text').text, stopword_list)) for child in
                         root.findall("./DOC")]
    else:
        # trec
        standard_xml_path = to_standard_xml(trec_sample_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text, token_lower_nostop_stem_list(merge_text(child.find('HEADLINE').text,child.find('TEXT').text), stopword_list)) for child in root.findall("./DOC")]

    return all_text_list

In [5]:
is_sample = 0
data_folder = 'collections/'
sample_txt_path = 'collections/sample.txt'
trec_sample_path = 'collections/trec.5000.xml'
sample_xml_path = 'collections/sample.xml'
stopword_filepath = 'englishST.txt'

all_text_list= [doc1_stemlist, doc2_stemlist, .......]

In [6]:
if(is_sample):
    all_text_list = xml_all_text(sample_xml_path,stopword_list,is_sample)
else:
    all_text_list = xml_all_text(trec_sample_path,stopword_list,is_sample)

generate inverted index

In [8]:
def inverted_index(all_text_list):
    '''
    return: # index dic[stem]={'doc_id':position}
    '''

    #get stem
    stem_set = set()
    for current_doc_tupple in all_text_list:
        for current_stem in current_doc_tupple[1]:
            stem_set.add(current_stem)

    index_dic = {}
    for current_stem in stem_set:
        current_stem_doc_position_dic = {}
        for current_doc_tupple in all_text_list:
            current_doc_id = current_doc_tupple[0]
            current_doc_text_list = current_doc_tupple[1]

            if (current_stem not in current_doc_text_list):
                continue

            position_list = [index+1 for index in range(len(current_doc_text_list))
                             if current_doc_text_list[index] == current_stem]
            current_stem_doc_position_dic[str(current_doc_id)] = position_list
        index_dic[current_stem] = current_stem_doc_position_dic
    return index_dic

In [9]:
index_dic = inverted_index(all_text_list)

In [10]:
def output_index(index_dic,file_path):
    sorted_index_dic_list = sorted(index_dic.items(),key=lambda x:x[0],reverse=False)
    with open(file_path,'w',encoding='utf-8') as file_1:
        for current_tupple in sorted_index_dic_list:
            current_stem = current_tupple[0]
            current_stem_doc_position_dic = current_tupple[1]
            file_1.write(current_stem)
            file_1.write(':\n')
            for current_doc_id,current_doc_id_position_list in current_stem_doc_position_dic.items():
                file_1.write('\t')
                file_1.write(str(current_doc_id))
                file_1.write(': ')
                current_doc_id_position_list_new = map(lambda x: str(x), current_doc_id_position_list)
                # print(','.join(current_doc_id_position_list_new))
                file_1.write(','.join(current_doc_id_position_list_new))
                file_1.write('\n')
            file_1.write('\n')

    return

output inverted dic

In [11]:
output_index_path = 'index.txt'
output_index(index_dic,output_index_path)

load inverted index from a file

In [12]:
def load_inverted_index(index_path):
    result_inverted_index = {}
    with open(index_path,'r') as f1:
        for line in f1:
            line = line.strip()
            if(line.endswith(':')):
                current_stem = line.replace(':','')
                result_inverted_index[current_stem] = {}
                continue
                
            if(len(line)==0):
                continue
                
            temp_split_list = line.split(': ')
            current_doc_id,str_position_list = temp_split_list[0],temp_split_list[1]
            current_position_list = [int(current_position) for current_position in str_position_list.split(',')]
            result_inverted_index[current_stem][current_doc_id] = current_position_list
    return result_inverted_index

In [13]:
inverted_index_path = 'index.txt'
loaded_inverted_index = load_inverted_index(inverted_index_path)

In [14]:
loaded_inverted_index

{'0': {'5000': [70, 323, 381],
  '5001': [88, 95],
  '5019': [268, 273],
  '5074': [161, 176, 185, 189, 198],
  '5087': [213, 218, 220],
  '5092': [18, 36, 43],
  '5097': [140],
  '5102': [292],
  '5107': [244],
  '5115': [517, 544, 564, 565, 590, 620],
  '5125': [29,
   100,
   208,
   210,
   212,
   218,
   227,
   234,
   247,
   249,
   260,
   279,
   286,
   288,
   301,
   312,
   325,
   338,
   351,
   364,
   377,
   390,
   392,
   400,
   417,
   430,
   443,
   453,
   469,
   471,
   482,
   484,
   488,
   492,
   497,
   519,
   523],
  '5154': [106],
  '5158': [114, 118],
  '5159': [141, 143, 153],
  '5162': [24],
  '5169': [193],
  '5218': [113, 118, 119, 130, 135, 182, 193, 266],
  '5223': [121, 123],
  '5266': [129],
  '5268': [62,
   360,
   362,
   366,
   373,
   377,
   426,
   464,
   466,
   533,
   541,
   554,
   563,
   565,
   568,
   577,
   578,
   580,
   593,
   627,
   629,
   633,
   668,
   677,
   682,
   693,
   695,
   719,
   769,
   782,
   79

In [19]:
def get_doc_id_set(current_inverted_dic):
    doc_id_set = set()
    for current_stem,current_doc_position_dic in current_inverted_dic.items():
        for current_doc_id in current_doc_position_dic.keys():
            doc_id_set.add(str(current_doc_id))
    return doc_id_set

def query_word(current_inverted_dic,current_word,is_not=0):
    current_word_stem = stem(current_word.lower())
    if(is_not):
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                doc_id_set = get_doc_id_set(current_inverted_dic)
                stem_doc_list = set(current_index_stem_position.keys())
                return list(doc_id_set.difference(stem_doc_list))
    else:
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                return list(current_index_stem_position.keys())
#     raise RuntimeError('Query not found.')
    return []

# OR
def union_list(a,b):
    result_list = list(set(a).union(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

# AND
def intersection_list(a,b):
    result_list = list(set(a).intersection(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

def probability_query(doc_word_pos1,doc_word_pos2,current_distance):
    result_list = []
    for current_docid_1,current_positionlist_1 in doc_word_pos1.items():
        if(current_docid_1 not in doc_word_pos2.keys()):
            continue
        current_positionlist_2 = doc_word_pos2[current_docid_1]
        i = 0
        j = 0

        while((i <= len(current_positionlist_1) - 1) and (j <= len(current_positionlist_2) - 1)):
            if(int(current_positionlist_1[i]) > int(current_positionlist_2[j]) + current_distance):
                j+=1
                continue
            elif(int(current_positionlist_1[i]) < int(current_positionlist_2[j]) - current_distance):
                i+=1
                continue
            else:
                result_list.append(current_docid_1)
                break
    return result_list

#phrase
def phrase_query(current_inverted_dic,query_phrase):
    phrase_list = [ stem(current_word.lower().strip()) for current_word in query_phrase.replace('"','').split()]
    result_list = probability_query(current_inverted_dic[phrase_list[0]],current_inverted_dic[phrase_list[1]],1)
    return result_list

def boolean_query(current_inverted_dic, current_query_word):
    is_NOT = 0
    
    #whether contains NOT
    if(current_query_word.startswith('NOT')):
        is_NOT = 1
        current_query_word = current_query_word.replace('NOT','').strip()
        
    if(current_query_word.startswith('"') and current_query_word.endswith('"')):
        result_list = phrase_query(current_inverted_dic,current_query_word)
    elif(current_query_word.startswith('#')):
        current_query_word_list = current_query_word.split('(')
        current_word_distance = int(current_query_word_list[0].replace('#','').strip())
        current_query_stem_list = [stem(temp_word.strip()) for temp_word in current_query_word_list[1].replace(')','').split(',')]
        result_list = probability_query(current_inverted_dic[current_query_stem_list[0]],current_inverted_dic[current_query_stem_list[1]],current_word_distance)
    else:
        result_list = query_word(current_inverted_dic,current_query_word)
    
    # if contains NOT, get the difference set of the result_list from the doc_id_set
    if(is_NOT):
        doc_id_set = get_doc_id_set(current_inverted_dic)
        result_list = list(doc_id_set.difference(set(result_list)))
    
    return result_list



tfidf

In [20]:
from collections import defaultdict
from math import log10

def tf_idf_weight(current_inverted_dic,query_phrase,is_stop=0,stop_word_path=None):
    #doc_tfidf['doc_id'] = tf_idf
    doc_tfidf = defaultdict(int)
    
    # N
    doc_number = len(get_doc_id_set(current_inverted_dic))
    
    if(is_stop == 0):
        stem_word_list = [ stem(current_query_word.strip().lower()) for current_query_word in query_phrase.split()]
    else:
        stopword_list = load_stopword(stop_word_path)
        stem_word_list = [ stem(current_query_word.strip().lower()) for current_query_word in query_phrase.split() if current_query_word.strip().lower() not in stopword_list]

    if(len(stem_word_list) == 0 or (len(list(set(stem_word_list).intersection(set(current_inverted_dic.keys())))) == 0 )):
        raise RuntimeError('All query word not found!')
    
    for current_stem in stem_word_list:
        
        #current_stem_doc_tf_count['doc_id'] = count
        current_stem_doc_tf_count = {}
        
        if(current_stem not in current_inverted_dic.keys()):
            continue
            
        for current_doc_id,current_doc_pos in current_inverted_dic[current_stem].items():
            current_stem_doc_tf_count[current_doc_id] = len(current_doc_pos)
        
        current_stem_df = len(current_inverted_dic[current_stem].keys())
        
        for current_doc_id,current_stem_doc_tf in current_stem_doc_tf_count.items():
            doc_tfidf[current_doc_id] += (1 + log10(current_stem_doc_tf)) * log10(doc_number/current_stem_df)
    
    sorted_doc_tfidf = sorted(doc_tfidf.items(), key = lambda x:x[1], reverse = True)
    
    return sorted_doc_tfidf

In [21]:
def output_query(query_result_dic,is_boolean=1):
    if(is_boolean):
        with open('results.boolean.txt','w',encoding='utf-8') as oq_file:
            for current_query_id,current_query_result_list in query_result_dic.items():
                for current_doc_id in current_query_result_list:
                    oq_file.write('%d 0 %d 0 1 0\n'%(int(current_query_id),int(current_doc_id)))
    else:
        with open('results.ranked.txt','w',encoding='utf-8') as oq_file:
            for current_query_id,current_query_result_list in query_result_dic.items():
                for current_docid_tfidf_tuple in current_query_result_list:
                    oq_file.write('%d 0 %d 0 %.4f 0\n'%(int(current_query_id),int(current_docid_tfidf_tuple[0]),current_docid_tfidf_tuple[1]))
                    
    return

In [22]:
query_file_list = ['queries.boolean.txt','queries.ranked.txt']
operator_list = ['AND','OR']

for query_file_path in query_file_list:
    #query_result_dic[query_id] = current_query_result_list
    query_result_dic = {}
    if(query_file_path.split('.')[1] == 'boolean'):
        with open(query_file_path,'r') as query_file:
            for current_query_line in query_file:

                query_result_list = []
                is_operator = 0
                current_query_line = current_query_line.strip()
                if(len(current_query_line) == 0):
                    continue
                current_query_id = current_query_line.split()[0] # get query id
                current_query = ' '.join(current_query_line.split()[1:]) # get query text

                #check whether contains any operator: 'AND' or 'OR'
                for current_operator in operator_list:
                    if(current_operator in current_query):
                        result_list = []
                        is_operator = 1
                        current_query_split_list = current_query.split(current_operator)
                        for current_query_word in current_query_split_list:
                            result_list.append(boolean_query(loaded_inverted_index,current_query_word.strip()))

                        #AND
                        if(current_operator == 'AND'):
                            query_result_list = intersection_list(result_list[0],result_list[1])
                        #OR
                        else:
                            query_result_list = union_list(result_list[0],result_list[1])
                #no operator in query
                if(not is_operator):
                    query_result_list = boolean_query(loaded_inverted_index,current_query)
                query_result_dic[str(current_query_id)] = query_result_list
                print(current_query_line)
                print(query_result_list)
            output_query(query_result_dic)
    elif(query_file_path.split('.')[1] == 'ranked'):
        with open(query_file_path,'r') as query_file:
            for current_query_line in query_file:
                current_query_id = current_query_line.split()[0] # get query id
                current_query = ' '.join(current_query_line.split()[1:]) # get query text

                current_result = tf_idf_weight(loaded_inverted_index,current_query.strip(),is_stop=1,stop_word_path=stopword_path)

                # with a maximum of 1000 results per query
                query_result_dic[str(current_query_id)] = current_result[0:1000]
            output_query(query_result_dic,is_boolean=0)

1 Happiness
['5016', '5130', '5239', '5247', '5262', '5263', '5297', '5396', '5445', '5448', '5452', '5460', '5467', '5470', '5477', '5547', '5590', '5716', '5727', '5833', '5943', '6359', '6404', '6438', '6445', '6448', '6451', '6468', '6473', '6475', '6560', '6638', '6640', '6660', '6770', '6796', '6836', '6914', '6975', '6987', '7181', '7292', '7343', '7365', '7460', '7472', '7529', '7560', '7569', '7620', '7662', '7664', '7679', '7683', '7687', '7688', '7691', '7720', '7846', '7875', '8019', '8049', '8080', '8184', '8185', '8234', '8321', '8332', '8679', '8732', '8792', '8914', '8955', '9025', '9029', '9036', '9038', '9048', '9060', '9062', '9063', '9082', '9091', '9186', '9275', '9278', '9390', '9441', '9589', '9612', '9636', '9739', '9745', '9819', '9822', '9827', '9969', '10035']
2 Edinburgh AND SCOTLAND
[]
3 income AND taxes
['5019', '5023', '5115', '5145', '5158', '5169', '5174', '5323', '5350', '5387', '5390', '5400', '5483', '5493', '5506', '5585', '5660', '5663', '5664', '5